In [ ]:
#title: "Financial anomaly detection with benford law"

#author: Erkan Çetinyamaç 

Veri Seti  IEEE-CIS tarafından oluşturulmuş bir sentetik data setidir. Fraud Detection problemleri üzerinde araştırma yapılması için oluşturulmuş bir data settir.

Link: https://www.kaggle.com/c/ieee-fraud-detection/data

In [ ]:
library(plyr)
library(dplyr)
library(ggplot2)
library(tidyverse)
library(benford.analysis)
library(rcompanion)
library(funModeling)


Verideki varsa Eksik Gözlemlerin Silinmesi.

In [ ]:
data<- train_transaction

In [ ]:
dim(data)

In [ ]:
colSums(is.na(data))

na.omit(data)

Popülasyondan örneklem çekimi.

In [ ]:
set.seed(2021)

df<-sample_n(data, 100000)

Veriyi indexlemek.

In [ ]:
index<- c(1:100000)

df$index<-index

İlgili Featureları alınıyor.

In [ ]:
df<-df[c("index","TransactionAmt","TransactionID","TransactionDT","ProductCD","card1","card2","card3","card4","card5" ,"card6")]


TransactionDT: timedelta from a given reference datetime (not an actual timestamp)
TransactionAMT: transaction payment amount in USD
ProductCD: product code, the product for each transaction
card1 - card6: payment card information, such as card type, card category, issue bank.

Veriyi Tanıyalım.


In [ ]:
glimpse(df)

In [ ]:

Verinin istatistiksel özellikleri

In [ ]:
summary(df)

profiling_num(df)

Verinin Görselleştirilmesi

In [ ]:
freq(df)

In [ ]:
plot_num(df)

In [ ]:
plotNormalHistogram(df$TransactionAmt)

range(df$TransactionAmt)

Benford

In [ ]:
lead_digit <- extract.digits(df$TransactionAmt, number.of.digits=1)
lead_digit <- lead_digit[,2]
hist(lead_digit, breaks=9)
ben <- benford(df$TransactionAmt, number.of.digits=2)
plot(ben)

ilk 2 digiti sırasıyla 4 ve 9 olan transactionların normalden çok daha fazla olduğu gözlemleniyor.


49 ile başlayan gözlemleri veri setinden çekiyoruz ve raporlamaya hazır hale getiriyoruz.

In [ ]:
lead_digit <- extract.digits(df$TransactionAmt, number.of.digits=2)
customer_check <- cbind(df,lead_digit) 

PossibleFraud<-customer_check[customer_check$data.digits ==49 ,] %>%
  arrange(index) %>% 
  select(index, TransactionAmt)

PossibleFraud